<a href="https://colab.research.google.com/github/tonycico0912/ChatGPT_Python/blob/main/%EA%B5%AD%EB%82%B4%EC%A3%BC%EC%8B%9D%EB%8D%B0%EC%9D%B4%ED%84%B0%EC%88%98%EC%A7%91.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install beautifulsoup4


In [4]:
import requests as rq  # requests 라이브러리를 rq라는 이름으로 가져옴
from bs4 import BeautifulSoup  # BeautifulSoup 라이브러리를 가져옴 : 이 라이브러리는 HTML을 파싱하고 정보를 추출하는데 사용됨

url = 'https://finance.naver.com/sise/sise_deposit.nhn'  # 스크래핑할 웹페이지의 URL을 정의
data = rq.get(url)  # requests의 get 함수를 사용하여 웹페이지의 HTML을 가져옴
data_html = BeautifulSoup(data.content)  # 가져온 HTML을 BeautifulSoup 객체로 변환하여 파싱할 수 있게 함

# 'div.subtop_sise_graph2 > ul.subtop_chart_note > li > span.tah' CSS 선택자를 사용하여 웹페이지에서 특정 요소를 선택
# select_one 함수는 CSS 선택자에 일치하는 첫 번째 요소를 반환
parse_day = data_html.select_one('div.subtop_sise_graph2 > ul.subtop_chart_note > li > span.tah').text

print(parse_day)  # 선택한 요소의 텍스트를 출력

import re  # re 모듈을 가져옴. re 모듈은 정규 표현식을 사용하여 문자열을 처리하는데 사용

# re.findall 함수를 사용하여 parse_day 문자열에서 숫자를 모두 찾음 '[0-9]+'는 하나 이상의 숫자를 의미하는 정규 표현식임
biz_day = re.findall('[0-9]+', parse_day)

# ''.join 함수를 사용하여 찾은 숫자들을 하나의 문자열로 합침
biz_day = ''.join(biz_day)

print(biz_day)  # 합친 문자열을 출력



  |  2023.07.06
20230706


In [5]:
import requests as rq  # 웹 요청을 보내기 위한 requests 라이브러리를 가져옴
from io import BytesIO  # 바이트 스트림을 처리하기 위한 io 라이브러리의 BytesIO를 가져옴
import pandas as pd  # 데이터 처리를 위한 pandas 라이브러리를 가져옴

gen_otp_url = 'http://data.krx.co.kr/comm/fileDn/GenerateOTP/generate.cmd'  # OTP를 생성하는 URL을 정의

# OTP 생성 요청에 필요한 파라미터를 정의
gen_otp_stk = {
    'mktId': 'STK',
    'trdDd': biz_day,
    'money': '1',
    'csvxls_isNo': 'false',
    'name': 'fileDown',
    'url': 'dbms/MDC/STAT/standard/MDCSTAT03901'
}

# 요청 헤더를 정의.  Referer 헤더는 요청이 어디서 시작되었는지를 나타냄
headers = {'Referer': 'http://data.krx.co.kr/contents/MDC/MDI/mdiLoader'}

# OTP 생성 URL에 POST 요청을 보냄.  요청의 본문에는 gen_otp_stk 딕셔너리를, 헤더에는 headers 딕셔너리를 사용
otp_stk = rq.post(gen_otp_url, gen_otp_stk, headers=headers).text

print(otp_stk)  # 생성된 OTP를 출력


PxldVNq82TWyLSb3hI2XSQB1aGl+5az065xdS2mm2KYRtSksuLS7Bnxpl86F7dAOkunw9BBwugQaSjGAcH15eZldSFIiFMVfhJGPzsNJfHMtBgM+EFJCxYg3zco1gIgRZqIo4cIzoURnTI8+MmkJ4m8vFLhSKmM794gFu+ThsO31lY4woqehX8j6OlXFDcfHdV4NbYo4+D2Rwcfj24VnU3Zpq3ik/Dyw3FdyOXhJkBI=
